# Learn OpenAI Whisper - Chapter 9
## Notebook 2 - PVS step 1: Processing audio files to LJSpeech format with Whisper and OZEN

This notebook complements the book [Learn OpenAI Whisper](https://a.co/d/1p5k4Tg).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wnomL0dxmU9CgPKIgazR8AocolEYjAe5)

This notebook represents the initial step in the 3-step personalized voice synthesis (PVS) process outlined in the chapter. This step takes an audio sample of the target voice as input and processes it into the LJSpeech dataset format. The notebook demonstrates how to use the OZEN Toolkit and OpenAI's Whisper to extract speech, transcribe it, and organize the data according to the LJSpeech structure. The resulting LJSpeech-formatted dataset, consisting of segmented audio files and corresponding transcriptions, serves as the input for the second step, "PVS step 2: Fine-tuning a discrete variational autoencoder using the DLAS toolkit," where a PVS model is fine-tuned using this dataset.

This notebook is based on the [OZEN Toolkit](https://github.com/devilismyfriend/ozen-toolkit) project. Given a folder of files or a single audio file, it will extract the speech, transcribe using Whisper and save in the LJ format (segmented audio files in WAV format in `wavs` folder, transcriptions in folders `train` and `valid`).

**NOTE**: The notebook stores the files using the following format.

`dataset/`
* ---├── `valid.txt`
* ---├── `train.txt`
* ---├── `wavs/`

`wavs/` directory must contain `.wav` files.

Example for `train.txt` and `valid.txt`:

* `wavs/A.wav|Write the transcribed audio here.`



## 1.	Cloning the OZEN Toolkit repository:

The following command clones the OZEN Toolkit repository from GitHub, which contains the necessary scripts and utilities for processing audio files:

In [ ]:
!git clone https://github.com/devilismyfriend/ozen-toolkit

## 2.	Installing required libraries

These following commands install the necessary libraries for audio processing, speech recognition, and text formatting:

In [ ]:
!pip -q install transformers
!pip -q install huggingface
!pip -q install pydub
!pip -q install yt-dlp
!pip -q install pyannote.audio

# RESTART SESSION

After installing the dependencies, restarting the session is recommended to ensure the installed packages are properly initialized.

In Google Colab, from the top menu, select `Runtime`, then `Restart session`.
<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/Restart_the_runtime_600x102.png" width=600>

In [ ]:
!pip -q install colorama
!pip -q install termcolor
!pip -q install pyfiglet

## 3.	Changing the working directory

The next command changes the working directory to the cloned ozen-toolkit directory:

In [ ]:
%cd ozen-toolkit

## 4.	Downloading a sample audio file

If you do not have an audio file for synthesis, this command downloads a sample audio file from the specified URL for demonstration purposes:

In [ ]:
# Download sample file
!wget -nv https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter01/Learn_OAI_Whisper_Sample_Audio01.mp3

## 5.	Uploading custom audio files

If you have your audio file, this code block allows users to upload their audio files to the Colab environment. It creates a directory in `/content/ozen-toolkit` to store the uploaded files and saves them in that directory:

In [ ]:
# CUSTOM_VOICE_NAME = "custom"

import os
from google.colab import files

custom_voice_folder = "./myaudiofile"

os.makedirs(custom_voice_folder, exist_ok=True)  # Create the directory if it doesn't exist

for filename, file_data in files.upload().items():
    with open(os.path.join(custom_voice_folder, filename), 'wb') as f:
        f.write(file_data)

%ls -l "$PWD"/{*,.*}

## 6.	Creating a configuration file
The following code section creates a configuration file named `config.ini` using the `configparser` library. It defines various settings such as the Hugging Face API key, Whisper model, device, diarization and segmentation models, validation ratio, and segmentation parameters:

In [ ]:
import configparser

# Create a new ConfigParser object
config = configparser.ConfigParser()

# Add the 'DEFAULT' section and set the options
config['DEFAULT'] = {
    'hf_token': '<HF_TOKEN>',
    'whisper_model': 'openai/whisper-medium',
    'device': 'cuda',
    'diaization_model': 'pyannote/speaker-diarization',
    'segmentation_model': 'pyannote/segmentation',
    'valid_ratio': '0.2',
    'seg_onset': '0.7',
    'seg_offset': '0.55',
    'seg_min_duration': '2.0',
    'seg_min_duration_off': '0.0'
}

# Write the configuration to a file
with open('config.ini', 'w') as configfile:
    config.write(configfile)

# Print the contents of the file
with open('config.ini', 'r') as configfile:
    print(configfile.read())

## 7.	Running the OZEN script

This command runs the ozen.py script with the sample audio file as an argument (or the file you uploaded).

# IMPORTANT:
`ozen.py` requires Hugging Face's `pyannote/segmentation` model. This is a gated model; you MUST request access before attempting to run the next cell. Thankfully, getting access is relatively straightforward and fast.

*   You must already have a Hugging Face account; if you do not have one, see the instructions in the notebook for chapter 3:  [LOAIW_ch03_working_with_audio_data_via_Hugging_Face.ipynb](https://colab.research.google.com/drive/1bIiGyv_YiTdq97a7KrowCceOrZlG2hXL#scrollTo=VCEKs-Y4wAYQ)
*   Visit https://hf.co/pyannote/segmentation to accept the user conditions.

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/HF_pyanonote_sementation_gated_model.JPG" width=600>

The script processes the audio file, extracts speech, transcribes it using Whisper, and saves the output in the LJSpeech format. The script saves the DJ format files in a folder called `ozen-toolkit/output/<audio file name + timestamp>/`. Here is an example of the expected file structure:
```
ozen-toolkit/output/
---├── Learn_OAI_Whisper_Sample_Audio01.mp3_2024_03_16-16_36/
------------------├── valid.txt
------------------├── train.txt
------------------├── wavs/
--------------------------├── 0.wav
--------------------------├── 1.wav
--------------------------├── 2.wav
```

In [ ]:
!python ozen.py Learn_OAI_Whisper_Sample_Audio01.mp3

#### Mount Google Drive (To save trained checkpoints and to load the dataset from)

## 8.	Mounting Google Drive

These lines mount the user's Google Drive to the Colab environment, allowing access to the drive for saving checkpoints and loading datasets:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## 9.	Copying the output to Google Drive

The following command copies the processed output files from the `ozen-toolkit/output` directory to your Google Drive.

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/ch09_2_Google_Colab_directory.JPG" width=600>


In [ ]:
%cp -r /content/ozen-toolkit/output/ /content/gdrive/MyDrive/

After running the cell, go to your Google Drive using a web browser, and you will see a directory called `output` with the DJ format dataset files in it.

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter09/images/ch09_2_Google_Drive_directory.JPG" width=500>

---
With our audio data now converted to the LJSpeech format, we are well-prepared to embark on the following critical stage of the PVS journey: fine-tuning a VPS model using the powerful DLAS toolkit. The notebook [LOAIW_ch09_3_Fine-tuning_PVS_models_with_DLAS.ipynb](LOAIW_ch09_3_Fine-tuning_PVS_models_with_DLAS.ipynb) will cover that process in detail. By leveraging the DLAS toolkit's comprehensive features and the structured LJSpeech dataset, we can create a personalized voice model that captures the unique characteristics of our target voice with remarkable accuracy and naturalness.